# Предсказываем стоимость мед страховки

Рассмотрим пример использования машинного обучения для домашней бухгалтерии. Применим машинное обучение для предсказания стоимости медицинской страховки для человека.

Данные для анализа:

https://docs.google.com/spreadsheets/d/1LVhrLj1khYNLlgtGPDecml5k2T7vsW0MlpC4voZNa4s?usp=drive_fs


# Загружаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df
df = pd.read_csv("insurance.csv", sep=",")


In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,False,True
1,18,1,33.770,1,0,1725.55230,False,False,True,False
2,28,1,33.000,3,0,4449.46200,False,False,True,False
3,33,1,22.705,0,0,21984.47061,False,True,False,False
4,32,1,28.880,0,0,3866.85520,False,True,False,False


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'charges',
       'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest'],
      dtype='object')

In [ ]:
X = df[['age', 'sex', 'bmi', 'children', 'smoker', 'region_northeast', 'region_northwest', 'region_southeast',
       'region_southwest']]
y = df[['charges']]


In [ ]:
X

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,False,False,False,True
1,18,1,33.770,1,0,False,False,True,False
2,28,1,33.000,3,0,False,False,True,False
3,33,1,22.705,0,0,False,True,False,False
4,32,1,28.880,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,False,True,False,False
1334,18,0,31.920,0,0,True,False,False,False
1335,18,0,36.850,0,0,False,False,True,False
1336,21,0,25.800,0,0,False,False,False,True


In [ ]:
y

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
137,22,1,25.175,0,0,False,True,False,False
399,18,0,38.170,0,0,False,False,True,False
160,42,0,26.600,0,1,False,True,False,False
1232,54,0,24.605,3,0,False,True,False,False
744,50,1,26.410,0,0,False,True,False,False
...,...,...,...,...,...,...,...,...,...
1091,55,0,29.830,0,0,True,False,False,False
1196,19,0,30.020,0,1,False,True,False,False
1300,45,1,30.360,0,1,False,False,True,False
187,30,0,30.900,3,0,False,False,False,True


In [ ]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
895,61,0,44.00,0,0,False,False,False,True
824,60,1,24.32,0,0,False,True,False,False
151,48,1,29.70,0,0,False,False,True,False
514,39,1,28.30,1,1,False,False,False,True
1215,18,1,39.14,0,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...
62,64,1,24.70,1,0,False,True,False,False
360,48,0,32.30,2,0,True,False,False,False
104,34,0,27.50,1,0,False,False,False,True
653,45,0,36.30,2,0,False,False,True,False


In [ ]:
y_train

,charges
137,2045.68525
399,1631.66830
160,21348.70600
1232,12479.70895
744,8827.20990
...,...
1091,11286.53870
1196,33307.55080
1300,62592.87309
187,5325.65100


In [ ]:
y_test

,charges
895,13063.88300
824,12523.60480
151,7789.63500
514,21082.16000
1215,12890.05765
...,...
62,30166.61817
360,10043.24900
104,5003.85300
653,8527.53200


## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
lr.coef_

array([[  251.48373333,  -166.50803241,   364.62532847,   459.47080174,
        23997.65536392,   351.50635992,   361.18454893,  -544.42384351,
         -168.26706535]])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание
lr.predict(X_test)

array([[18188.69866717],
       [11124.33205133],
       [ 9162.60312618],
       [31222.05701021],
       [ 5956.08443063],
       [ 1253.55157366],
       [11067.49006125],
       [30354.65614181],
       [27438.45237522],
       [ 2854.0411134 ],
       [11186.29323028],
       [ 3326.42449694],
       [11693.79402368],
       [ 6492.57091829],
       [32007.76211002],
       [ 9413.56767397],
       [ 7156.4609499 ],
       [  610.26814689],
       [36919.78193208],
       [17226.49924221],
       [ 9098.8793038 ],
       [ -121.76498256],
       [12358.88817899],
       [ 6995.43257734],
       [ 8177.92007142],
       [26278.27363306],
       [11026.63741589],
       [ 3276.73740331],
       [23492.89240642],
       [31791.99284371],
       [32512.9616075 ],
       [35604.16641897],
       [12949.82053695],
       [  509.94247557],
       [11553.52946126],
       [15721.69488759],
       [30991.17611997],
       [13264.5101307 ],
       [ 4795.62820117],
       [ 4926.94729265],


In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

32289564.99752574

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

37634358.681340106

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([[27147.04315901]])